In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [104]:
from proj1_helpers import *
from implementations import *
from data_helpers import *

In [105]:
#DATA_TRAIN_PATH = '/content/drive/My Drive/ML_Project1/data/train.csv'
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
tx, mean, std = standardize(tX,0)
y, tx = build_model_data(tx,y)

In [106]:
ratio = 0.9

train_x, train_y, test_x, test_y = split_data(tx, y, ratio)

In [107]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((225000, 31), (225000,), (25000, 31), (25000,))

## Do your thing crazy machine learning thing here :) ...

In [108]:
lambdas = np.logspace(-5,0,20)

loss_min = np.inf

for _,lambda_ in enumerate(lambdas):

    weights,loss_training = ridge_regression(train_y, train_x, lambda_)
    loss_validation = compute_loss(test_y, test_x, weights)
    print("Lambda={l}, Training Loss ={tl}, Validation Loss ={vl}".format(
               l=lambda_, tl=loss_training, vl=loss_validation))
    
    if (loss_validation < loss_min):
        
        loss_min = loss_validation
        lambda_min = lambda_
        weights_star = weights
print(lambda_min)

Lambda=1e-05, Training Loss =0.3396167585167065, Validation Loss =0.34173700972634036
Lambda=1.8329807108324375e-05, Training Loss =0.3396840033693898, Validation Loss =0.3418162262842967
Lambda=3.359818286283781e-05, Training Loss =0.33973150072617786, Validation Loss =0.3418707869521441
Lambda=6.158482110660267e-05, Training Loss =0.3397623943876683, Validation Loss =0.3419046892922824
Lambda=0.00011288378916846884, Training Loss =0.33978189746153864, Validation Loss =0.3419244274296717
Lambda=0.00020691380811147902, Training Loss =0.33979441760124673, Validation Loss =0.34193547459004403
Lambda=0.000379269019073225, Training Loss =0.3398032489901154, Validation Loss =0.3419416108620644
Lambda=0.0006951927961775605, Training Loss =0.3398117519090294, Validation Loss =0.341945979247488
Lambda=0.0012742749857031334, Training Loss =0.33982634432465086, Validation Loss =0.34195409747723265
Lambda=0.002335721469090121, Training Loss =0.3398647115150276, Validation Loss =0.3419821829490181

## Generate predictions and save ouput in csv format for submission:

In [91]:
DATA_TEST_PATH = 'data/test.csv' # TODO: download train data and supply path here 
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [92]:
_, tx_test = build_model_data(tX_test,ytest)

In [93]:
OUTPUT_PATH = 'data/submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights_star, tx_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)